# Solving Linear Optimization Problems in Python with CVXOPT

# Step 1 - Defining the Problem

In a decision problem, we have three key components: 
1. The decision variables. These are the variables we can and will control to reach the outcome we want. An example of a decision variable is the store price of a new product we are developing.
2. The objective function. This represents the value we are affecting by controlling the decision variable. In a decision problem, we will either try to maximize a value (such as profit) or minimize a value (such as cost).
3. The constraints. These are the factors that limit our range of feasible solutions. A constraint can be ethical, practical, or physical. For example, we might not be able to apply a certain medical treatment to pregnant women due to health risks, or we might have budget constraints that limit how many units we can produce. Because machines don't understand the real-world context of our decision problems, we need to express these constraints in algebraice form. Constraints can take the form of equations or inequalities.

A linear optimization, or linear programming, problem, is a subset of decision problems in which the objective function and the constraints are all linear combinations of the decision variables. If we have decision variables $x_1, x_2, ..., x_n$, then our objective function may look something like:
 - $a_1x_1+a_2x_2+...+a_nx_n=c$

and our constraints something like:
 - $b_1x_1+b_2x_2+...+b_nx_n \leq b_1$
 - $c_1x_1+c_2x_2+...+c_nx_n = b_2$

This requires three important assumptions:
 - Proportionality - the effect of changing the value of a decision variable is always linear, never marginal
 - Additivity - there are no interaction effects; changing one decision variable does not influence the strength of another decision variable
 - Certainty - there is no noise in the parameters or the constraints

We will consider two types of linear programming - general linear programming and integer programming. Each requires one more assumption:
 - In general linear programming, the decision variables can be any real number
 - In integer programming, the decision variables must take integer values.

It is important to understand how these assumptions apply to your decision problem, as they influence what the proper solution is. If we know all our assumptions are satisfied, we can move on to solving the problem.

# Step 2 - Build the Optimization Model

## 1. Basic Linear Programming


### Install the libraries:

We'll be using the CVXOPT library, as it is not only convenient to use, but its structure highlights the uses of matrices, which helps us intuit the structure of our decision problem. 

In [ ]:
from cvxopt import matrix, solvers
import numpy  

### Build the matrices

Let's say we have $n$ decision variables and $m$ constraints. We have the following matrices:
* $A$, which is an $m \times n$ matrix. This represents the coefficients of our constraints.

  $A =
 \begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m,1} & a_{m,2} & \cdots & a_{m,n}
 \end{pmatrix}$
* $b$, which is an $m \times 1$ vector. This represents the maximum values given to us by our constraints.

  $b =
 \begin{pmatrix}
  b_{1}  \\
  b_{2}  \\
  \vdots   \\
  b_{m}
 \end{pmatrix}$
* $c$, which is an $n \times 1$ vector. This represents the coefficients in our objective function.

  $c =
 \begin{pmatrix}
  c_{1}  \\
  c_{2}  \\
  \vdots   \\
  c_{n}
 \end{pmatrix}$

* $x$, which is an $n \times 1$ vector. This represents our decision variables.

  $x =
 \begin{pmatrix}
  x_{1}  \\
  x_{2}  \\
  \vdots   \\
  x_{n}
 \end{pmatrix}$

These four matrices relate to each other in the following ways:
* $Ax \leq b$ represents our constraints. Looking at the matrices themselves, we get

  $\begin{pmatrix}
  a_{1,1} & a_{1,2} & \cdots & a_{1,n} \\
  a_{2,1} & a_{2,2} & \cdots & a_{2,n} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  a_{m,1} & a_{m,2} & \cdots & a_{m,n}
 \end{pmatrix}\begin{pmatrix}
  x_{1}  \\
  x_{2}  \\
  \vdots   \\
  x_{n}
 \end{pmatrix} \leq 
 \begin{pmatrix}
  b_{1}  \\
  b_{2}  \\
  \vdots   \\
  b_{m}
 \end{pmatrix}$

 So, for row $i$, we get

 $a_{i,1}x_1 + a_{i,2}x_2 +...+ a_{i,2}x_2\leq b_i$

 Which is identical to the syntax we saw earlier. Notice that we specifically say that we want to be less than or equal to $b_i$. This is because CVXOPT specifically stays at or below the constraint values. This seems limiting but is actually quite simple to work around. For example, if our constraint $b_j$ is a minimum, meaning

  $a_{j,1}x_1  + a_{j,2}x_2 +... +a_{j,2}x_2\geq b_j$ 

  we simply make sure to multiply the coefficients of $A$ at row $j$ by $-1$ and to compare it to $-b_j$, so we get

  $-a_{j,1}x_1  -a_{j,2}x_2 -... -a_{j,2}x_2\leq -b_j$

* $c^T \times x$ represents our objective function, as 

  $\begin{pmatrix}
  c_{1} & c_{2} ... c_{n} \end{pmatrix} \times \begin{pmatrix}
  x_{1}  \\
  x_{2}  \\
  \vdots   \\
  x_{n} \end{pmatrix} = c_1x_1+c_2x_2+...+c_nx_n$

  Again, this is similar to the syntax we saw before for the objective function.

It's important to understand how the variables affect the structure of the matrices. When setting up your matrices, just remember that $m$ represents the number of constraints, $n$ represents the number of variables, and that when multiplying two matrices, where the left matrix is $m_1\times n_1$ and the right matrix is $m_2 \times n_2$, $n_1$ must equal $m_2$. Remembering how matrix multiplication works can let you work backwards to the shape of each matrix, as long as you know how they relate to each other.

It's also important to define your function clearly beforehand, as it helps you set up the matrices. You want to make sure you're consistent with where you're putting your coefficients. In real life, decision variables and constraints aren't inherently ordered, so you need to manually put all the coefficients for the $jth$ constraint in the $jth$ row of $A$, and put the maximum value on the $jth$ entry of $b$.

Now, after all that explanation, we simply have three lines of code:

In [ ]:
# Note that this code will not run as is, as the entries in the matrices are just placeholders
# When you set up your matrices, simply put the corresponding numbers in place of the variables

A = matrix([ [a11, a12, a13, a14], [a21, a22, a23, a24] ])
b = matrix([ b1, b2, b3, b4 ])
c = matrix([ c1, c2 ])

$b$ and $c$ are both column vectors, so they're straightforward to define. Simply use the function cvxopt.matrix() - or just matrix(), as we imported the function from the CVXOPT library - and put in a list of values. $b$ requires a list of length $m$ and $c$ requires a list of length $n$.

$A$ is slightly more difficult as it can be two-dimensional, which calls for a list of lists. Just remember that each list in $A$ has to be the same length as $b$, as in $m$ elements long, and that the number of lists must equal the length of $c$, so $n$ lists. This translates to $n$ lists of length $m$, with each list representing a column vector, resulting in an $m \times n$ matrix.  

You'll notice that we didn't set up a variable for vector $x$, and that's because we don't have to. $x$ is a vector of variable values, and is taken care of by CVXOPT.

Now that we've set up our matrices, we can actually solve the problem. For the sake of actually solving a problem and looking at the output, let's define our matrices with actual numbers.

Let's say we have two decision variables, $x_1$ and $x_2$. Our objective function is to maximize $x_1 + 2x_x$. Our constraints are:
 * $2x_1 + 11x_2 \leq 58$
 * $2x_1 + 5x_2 \geq 24$
 * $x_1 \leq 8$
 * $x_2 \geq 3$

### Run the solver

In [ ]:
A = matrix([ [2.0, -2.0, 1.0, 0], [11.0, -5.0, 0, -1.0] ])
b = matrix([ 58.0, -24.0, 8.0, -3.0 ])
c = matrix([ -1.0, -2.0 ])

sol = solvers.lp(c, A, b)

     pcost       dcost       gap    pres   dres   k/t
 0: -1.2114e+01 -6.3635e+01  2e+01  0e+00  3e+00  1e+00
 1: -1.3255e+01 -2.2455e+01  3e+00  3e-17  5e-01  2e-01
 2: -1.5543e+01 -1.6576e+01  3e-01  2e-16  6e-02  5e-02
 3: -1.5634e+01 -1.5649e+01  4e-03  6e-17  9e-04  7e-04
 4: -1.5636e+01 -1.5636e+01  4e-05  3e-16  9e-06  7e-06
 5: -1.5636e+01 -1.5636e+01  4e-07  7e-17  9e-08  7e-08
Optimal solution found.


### Examine the outputs

Let's take a look at the output of cvx.solvers.lp()

In [ ]:
sol

{'dual infeasibility': 3.9166606476405134e-08,
 'dual objective': -11.00000035804455,
 'dual slack': 1.4530691843169395e-08,
 'gap': 2.4637363207829533e-08,
 'iterations': 5,
 'primal infeasibility': 6.389899865434352e-17,
 'primal objective': -10.999999989893329,
 'primal slack': 3.7206016348342837e-10,
 'relative gap': 2.2397602936787323e-09,
 'residual as dual infeasibility certificate': None,
 'residual as primal infeasibility certificate': None,
 's': <4x1 matrix, tc='d'>,
 'status': 'optimal',
 'x': <2x1 matrix, tc='d'>,
 'y': <0x1 matrix, tc='d'>,
 'z': <4x1 matrix, tc='d'>}

What we have is a dictionary several key-value pairs, most of which we won't go into. Let's focus on the key 'x', whose value is a $2\times 1$ vector, or generally, an $n \times 1$ vector. As you might guess, this represents the values of our decision variables. Specifically, these are the optimal values that our model calculated.

In [ ]:
print(sol['x'])

[ 8.00e+00]
[ 3.82e+00]



This tells us that, given the constraints, our optimal values are $x_1=8$ and $x_2=3.82$. To see what objective function value that gives us, we multiply the vector $c$ by these results.

In [ ]:
print(numpy.matmul(numpy.transpose(c),sol['x']))

[[-15.63636344]]


This gives us a negative value because we were minimizing $-c \times x$, rather than maximizing $c \times x$. Our actual optimal value is 11. Also, just to check how our solution compares to our constraints, let's multiply $b$ by $x$.

In [ ]:
print(numpy.matmul(A,sol['x']))

[[ 42.99999998]
 [-24.99999998]
 [  4.99999999]
 [ -3.        ]]


Recall that the values of $b$ were $58, -24, 8,$ and $ -3$, so our values are less than or equal to our constraints. Remember that our second constraint was $2x_1 + 5x_2 \geq 24$. What we used in our optimization problem was technically $-2x_1 + -5x_2 \leq -24$, which is equivalent. Remember that cvxopt.solvers.lp() will treat $b$ as a vector of maximum values for $Ax$, and rewrite your constraints as needed.

Another key fact to remember is that cvxopt.solvers.lp() will always solve for the minimum value of the objective function. This is fine, as we just need to use $-c$ to solve for the maximum value.

Perhaps the trickiest part of setting up a linear optimization problem is setting up the constraints properly. We are ironically constrained to writing our constraints in the form $A_ix \leq b_i$, in that the left side of the inequality must be less than the right side. This makes it tricky to implement certain constraints, especially ones that related decision variables to each other such as $x_1 = x_2$. 

However, we can use logic to recreate these. For example, $x_1 = x_2$ can be rewritten as a pair of constraints, $x_1 - x_2 \leq 0$ and $ -x_1 + x_2 \leq 0$. If you think mathematically about your constraints, you should be able to rewrite them into the format you need.

## 2. Integer programming:

Now that we've covered all the basic theory behind linear programming, we can move on the integer programming fairly easily. As we said before, integer programming is just linear programming where the decision variables must take on integer values. To build an integer programming model, we'll use the CVXPY library.

### Install the libraries:

In [ ]:
from cvxopt import glpk

### Build the matrices:

Let's use the same matrices as before and see how our solutions differ when our decision variables can only be integers.

In [ ]:
A = matrix([ [2.0, -2.0, 1.0, 0], [11.0, -5.0, 0, -1.0] ])
b = matrix([ 58.0, -24.0, 8.0, -3.0 ])
c = matrix([ -1.0, -2.0 ])

### Run the solver

Solving integer programming using CVXOPT works a little differently than normal linear programming. Rather than running cvxopt.solvers.lp(), we use cvxopt.glpk.ilp(), as this allows us to set the decision variables to integers.

The function parameter I = set([]) allows you to select the indices of the variables you want to set as integers. As we have two variables and want to set both as integers, we'll say I = set([0, 1]), denoting that x[0] and x[1], which refer to $x_1$ and $x_2$, respectively, can only be integers in our solution.

The output of the function is a tuple with two values. The first tells us the status of the optimization problem, whether it converged or not. The second gives us the vector $x$. We'll capture both so we can look at them.

In [ ]:
(status, x) = glpk.ilp(c, A, b, I = set([0,1]))

### Examine the outputs

First, let's look at the status message.

In [ ]:
print(status)

optimal


The status is "optimal", which tells us that the model was able to find an optimal solution. Keep in mind that not all decision problems will have an optimal solution. There are some issues with glpk which can cause your Python notebook to disconnect if an integer programming problem cannot converge, so be aware of that.

Next, the optimal decision variables.

In [ ]:
print(x[0], x[1])

7.0 4.0


Our previous values, using general linear programming, were 8 and 3.82, respectively, so switching to intger programming can affect the optimal value of a decision variable even if it was already an intger value. This makes sense, as the two variables do not exist in a vacuum. Changing the value of one can affect the feasible values of the other, as they have to fit the constraints.

Let's see how our objective function changed.

In [ ]:
print(-sum(c.T*x))

15.0


Our previous optimal value was about 15.64, so our optimal value decreased. This also makes sense, as adding more constraints, in this case $x_1 \in {0,1}$ and $x_2 \in {0, 1}$ can reduce the feasible region for your decision variables, and thus reduce your maximum or increase your minimum.

## Step 3 - Check your work

CVXOPT is a very useful tool for solving linear optimization problems. But more important than knowing how to use CVXOPT is knowing how to intuit the problem you are trying to solve. To properly model your decision problem, you must check your assumptions, have a solid grasp of the constraints that limit your decision variables, and use your knowledge of mathematics to guide yourself through the process. 

Remember that the machine can't do everything for you. Machines lack context, which is so vital when handling real-world scenarios. It is up to you to translate the real-world constraints into code, and to translate them you must first understand them, as well as how to explain them to your machine.